In [ ]:
import torch
from transformers import ViTFeatureExtractor, ViTForImageClassification
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import pandas as pd
import os
import wandb
from tqdm import tqdm
from functools import partial
from sklearn.model_selection import StratifiedKFold

In [ ]:
'''from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
secret_value = user_secrets.get_secret("wandb_key")
wandb.login(key=secret_value)

wandb.init(project="stanford cars vit")'''

Load feature extractor and pretrained vision transformer

In [ ]:
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')

In [ ]:
import os
import pandas as pd
import numpy as np
import joblib
#from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from tqdm import tqdm

In [ ]:
df = pd.read_csv('../input/pillimages/final_datatable.csv')

In [ ]:
dir_list1 = os.listdir("../input/pillimages/images/images")
dir_list2 = os.listdir("../input/pillimages/images2/images2")

imagesFolderLen = len(dir_list1)
dir_list = [""]*(len(dir_list1)+len(dir_list2))
check_list = dir_list1+dir_list2

for d in range(len(dir_list1)):
    dir_list[d] = "images/"+dir_list1[d]

for d in range(len(dir_list2)):
    dir_list[d+imagesFolderLen] = "images2/"+dir_list2[d]

#print(dir_list)
print(len(dir_list))
print(len(check_list))
print(check_list[:10])

In [ ]:
set_dir_list1 = set(dir_list1)
set_dir_list2 = set(dir_list2)
set_check_list = set(check_list)
print(len(set_dir_list1))
print(len(set_dir_list2))
if "B4CH0R9B7PEQ6GORRX-8XWOL-_G7W9_.JPG" in set_dir_list2:
    print("True")

In [ ]:
df["Image"] = df["Image"].apply(lambda x: x.split("/"))
df["Image"] = df["Image"].apply(lambda x: x[2:])
df["Image"] = df["Image"].apply(lambda x: "".join(x))
df["Image"]

In [ ]:
df = df[df["Image"].str.contains(".JPG")==True]
df

In [ ]:
df = df[df.Image.isin(set_check_list)]
df

In [ ]:
df['Image'] = df['Image'].apply(lambda a: "images/images/"+a if a in set_dir_list1 else "images2/images2/"+a)
df

In [ ]:
def append_ext(fn):
    return "../input/pillimages/" + fn

In [ ]:
df['Image'] = df['Image'].apply(append_ext)

In [ ]:
counts = df['Name'].value_counts()

res = df[~df['Name'].isin(counts[counts < 18].index)]

In [ ]:
len(res)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
res['Name'] = le.fit_transform(res['Name'])
res

In [ ]:
len(le.classes_)

In [ ]:
#from sklearn.model_selection import train_test_split
y = res["Name"]
res = res.drop("Name", axis = 1)

In [ ]:
paths = res["Image"]
#_train, X_test, y_train, y_test = train_test_split(paths, y, test_size=0.25, random_state=11)

In [ ]:
#X_train = X_train.reset_index()
#X_test = X_test.reset_index()

In [ ]:
import pandas as pd
import joblib
import numpy as np
import torch
import random
import albumentations
import matplotlib.pyplot as plt
import argparse
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
from PIL import Image
from tqdm import tqdm
from torchvision import models as models
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

In [ ]:
def seed_everything(SEED=42):
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.benchmark = True

SEED=42
seed_everything(SEED=SEED)
device = ('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Computation device: {device}")

In [ ]:
num_epochs = 10
lr = 2e-4
eval_steps = 100
record_steps= 10
save_checkpoint = 5

#wandb.config.update({"lr": lr, "num_epochs": num_epochs})

model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224', num_labels=230,
                                                   id2label={str(i): c for i, c in enumerate(le.classes_)},
                                                   label2id={c: str(i) for i, c in enumerate(le.classes_)},
                                                   ignore_mismatched_sizes=True).to(device)

optim = torch.optim.AdamW(model.parameters(), lr=lr)

# negative log likelihood -> multi-class classification
training_stats_step = 10

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
# image dataset module
class NaturalImageDataset(Dataset):
    def __init__(self, path, labels, tfms=None):
        self.X = path
        self.y = labels
        # apply augmentations
        if tfms == 0: # if validating
            self.aug = albumentations.Compose([
                albumentations.Resize(224, 224, always_apply=True),
                albumentations.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225], always_apply=True)
            ])
        else: # if training
            self.aug = albumentations.Compose([
                albumentations.Resize(224, 224, always_apply=True),
                albumentations.HorizontalFlip(p=1.0),
                albumentations.ShiftScaleRotate(
                    shift_limit=0.3,
                    scale_limit=0.3,
                    rotate_limit=30,
                    p=1.0
                ),
                albumentations.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225], always_apply=True)
            ])
    def __len__(self):
        return (len(self.X))

    def __getitem__(self, i):
        image = Image.open(self.X.iloc[i])
        image = self.aug(image=np.array(image))['image']
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        label = self.y.iloc[i]
        return torch.tensor(image, dtype=torch.float), torch.tensor(label, dtype=torch.long)

In [ ]:
#validation function
def validate(model, test_dataloader):
    print('Validating')
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(test_dataloader), total=int(len(test_data)/test_dataloader.batch_size)):
            data, target = data[0].to(device), data[1].to(device)
            #loss = criterion(outputs, target)

            outputs = model(pixel_values=data, labels=target)
            #loss = criterion(outputs, target)
            loss = outputs[0]

            val_running_loss += loss.item()
            #_, preds = torch.max(outputs.data, 1)
            val_running_correct += (torch.argmax(outputs["logits"], dim=1) == target).sum().item()

        val_loss = val_running_loss/len(test_dataloader.dataset)
        val_accuracy = 100. * val_running_correct/len(test_dataloader.dataset)
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}')

        return val_loss, val_accuracy

In [ ]:
# training function
def fit(model, train_dataloader):
    print('Training')
    model.train()
    train_running_loss = 0.0
    train_running_correct = 0
    for i, data in tqdm(enumerate(train_dataloader), total=int(len(train_data)/train_dataloader.batch_size)):
        data, target = data[0].to(device), data[1].to(device)
        optim.zero_grad()
        outputs = model(pixel_values=data, labels=target)
        #loss = criterion(outputs, target)
        loss = outputs[0]
        train_running_loss += loss.item()
        #_, preds = torch.max(outputs.data, 1)
        train_running_correct += (torch.argmax(outputs["logits"], dim=1) == target).sum().item()
        loss.backward()
        optim.step()

    train_loss = train_running_loss/len(train_dataloader.dataset)
    train_accuracy = 100. * train_running_correct/len(train_dataloader.dataset)

    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}")

    return train_loss, train_accuracy

In [ ]:
class EarlyStopping():
    def __init__(self, tolerance=5, min_delta=0):

        self.tolerance = tolerance
        self.min_delta = min_delta
        self.counter = 0
        self.early_stop = False

    def __call__(self, train_loss, validation_loss):
        if (validation_loss - train_loss) > self.min_delta:
            self.counter +=1
            if self.counter >= self.tolerance:
                self.early_stop = True

In [ ]:
early_stopping = EarlyStopping(tolerance=5, min_delta=10)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, matthews_corrcoef

In [ ]:
skf = StratifiedKFold(n_splits=3)
precision = []
recall = []
f1 = []
mcc = []
val_min = np.inf
for _, (train_index, test_index) in enumerate(skf.split(paths, y)):

    X_train = paths.iloc[train_index]
    y_train = y.iloc[train_index]

    X_test = paths.iloc[test_index]
    y_test = paths.iloc[test_index]

    train_data = NaturalImageDataset(X_train, y_train, tfms=1)
    test_data = NaturalImageDataset(X_test, y_test, tfms=0)

    # dataloaders
    trainloader = DataLoader(train_data, batch_size=16, shuffle=True)
    testloader = DataLoader(test_data, batch_size=16, shuffle=False)

    train_loss , train_accuracy = [], []
    val_loss , val_accuracy = [], []
    start = time.time()
    for epoch in range(10):
        print(f"Epoch {epoch+1} of {10}")
        train_epoch_loss, train_epoch_accuracy = fit(model, trainloader)
        val_epoch_loss, val_epoch_accuracy = validate(model, testloader)
        train_loss.append(train_epoch_loss)
        train_accuracy.append(train_epoch_accuracy)
        val_loss.append(val_epoch_loss)
        val_accuracy.append(val_epoch_accuracy)
        if(val_epoch_loss < val_min):
          val_min = val_epoch_loss
          torch.save(model.state_dict(), 'vit_model.pt')
          print('Detected network improvement, saving current model')
          # early stopping
          early_stopping(train_epoch_loss, val_epoch_loss)
          if early_stopping.early_stop:
            print("We are at epoch:", epoch + 1)
            break

    end = time.time()
    print(f"{(end-start)/60:.3f} minutes")

    model.load_state_dict(torch.load('./vit_model.pt'))
    model.eval()
    labels = []
    y_true = []
    with torch.no_grad():
        for i, data in tqdm(enumerate(testloader), total=int(len(test_data)/testloader.batch_size)):
                data, target = data[0].to(device), data[1].to(device)
                outputs = model(pixel_values=data, labels=target)
                #loss = criterion(outputs, target)
                loss = outputs[0]
                #_, preds = torch.max(outputs.data, 1)
                preds = torch.argmax(outputs["logits"], dim=1)
                labels.append(preds.to('cpu').numpy())
                y_true.append(target.to('cpu').numpy())

    y_pred = [item for sublist in labels for item in sublist]
    y_test =  [item for sublist in y_true for item in sublist]

    precision.append(precision_score(y_test, y_pred, average='macro'))
    recall.append(recall_score(y_test, y_pred, average = 'macro'))
    f1.append(f1_score(y_test, y_pred, average = 'macro'))
    mcc.append(matthews_corrcoef(y_test, y_pred))

    #accuracy plots
    plt.figure(figsize=(10, 7))
    plt.plot(train_accuracy, color='green', label='train accuracy')
    plt.plot(val_accuracy, color='blue', label='validataion accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

    # loss plots
    plt.figure(figsize=(10, 7))
    plt.plot(train_loss, color='orange', label='train loss')
    plt.plot(val_loss, color='red', label='validataion loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [ ]:
print("Precision", sum(precision) / len(precision))
print("Recall", sum(recall) / len(recall))
print("F1", sum(f1) / len(f1))
print("MCC", sum(mcc) / len(mcc))

In [ ]:
#train_data = NaturalImageDataset(X_train, y_train, tfms=1)
#est_data = NaturalImageDataset(X_test, y_test, tfms=0)

# dataloaders
#trainloader = DataLoader(train_data, batch_size=16, shuffle=True)
#testloader = DataLoader(test_data, batch_size=16, shuffle=False)

In [ ]:
'''def seed_everything(SEED=42):
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.benchmark = True

SEED=42
seed_everything(SEED=SEED)
device = ('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Computation device: {device}")'''

In [ ]:
'''num_epochs = 10
lr = 2e-4
eval_steps = 100
record_steps= 10
save_checkpoint = 5

#wandb.config.update({"lr": lr, "num_epochs": num_epochs})

model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224', num_labels=230,
                                                   id2label={str(i): c for i, c in enumerate(le.classes_)},
                                                   label2id={c: str(i) for i, c in enumerate(le.classes_)},
                                                   ignore_mismatched_sizes=True).to(device)

optim = torch.optim.AdamW(model.parameters(), lr=lr)

# negative log likelihood -> multi-class classification
training_stats_step = 10'''

In [ ]:
#len(X_train), len(X_test)

In [ ]:
# optimizer
#optimizer = optim.Adam(model.parameters())
# loss function
#criterion = nn.CrossEntropyLoss()

In [ ]:
'''#validation function
def validate(model, test_dataloader):
    print('Validating')
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(test_dataloader), total=int(len(test_data)/test_dataloader.batch_size)):
            data, target = data[0].to(device), data[1].to(device)
            #loss = criterion(outputs, target)

            outputs = model(pixel_values=data, labels=target)
            #loss = criterion(outputs, target)
            loss = outputs[0]

            val_running_loss += loss.item()
            #_, preds = torch.max(outputs.data, 1)
            val_running_correct += (torch.argmax(outputs["logits"], dim=1) == target).sum().item()

        val_loss = val_running_loss/len(test_dataloader.dataset)
        val_accuracy = 100. * val_running_correct/len(test_dataloader.dataset)
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}')

        return val_loss, val_accuracy'''

In [ ]:
'''train_loss , train_accuracy = [], []
val_loss , val_accuracy = [], []
start = time.time()
val_min = np.inf
for epoch in range(10):
    print(f"Epoch {epoch+1} of {10}")
    train_epoch_loss, train_epoch_accuracy = fit(model, trainloader)
    val_epoch_loss, val_epoch_accuracy = validate(model, testloader)
    train_loss.append(train_epoch_loss)
    train_accuracy.append(train_epoch_accuracy)
    val_loss.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)
    if(val_epoch_loss < val_min):
      val_min = val_epoch_loss
      torch.save(model.state_dict(), 'vit_model.pt')
      print('Detected network improvement, saving current model')

end = time.time()
print(f"{(end-start)/60:.3f} minutes")'''

In [ ]:
'''# accuracy plots
plt.figure(figsize=(10, 7))
plt.plot(train_accuracy, color='green', label='train accuracy')
plt.plot(val_accuracy, color='blue', label='validataion accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# loss plots
plt.figure(figsize=(10, 7))
plt.plot(train_loss, color='orange', label='train loss')
plt.plot(val_loss, color='red', label='validataion loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()'''

In [ ]:
#model.load_state_dict(torch.load('./vit_model.pt'))

In [ ]:
'''model.eval()
labels = []
y_true = []
with torch.no_grad():
    for i, data in tqdm(enumerate(testloader), total=int(len(test_data)/testloader.batch_size)):
            data, target = data[0].to(device), data[1].to(device)
            outputs = model(pixel_values=data, labels=target)
            #loss = criterion(outputs, target)
            loss = outputs[0]
            #_, preds = torch.max(outputs.data, 1)
            preds = torch.argmax(outputs["logits"], dim=1)
            labels.append(preds.to('cpu').numpy())
            y_true.append(target.to('cpu').numpy())'''

In [ ]:
#y_pred = [item for sublist in labels for item in sublist]
#y_test =  [item for sublist in y_true for item in sublist]

In [ ]:
from sklearn.metrics import classification_report
#print(classification_report(le.inverse_transform(y_test), le.inverse_transform(y_pred)))

In [ ]:
from sklearn.metrics import matthews_corrcoef
#print(matthews_corrcoef(y_test, y_pred))

In [ ]:
from sklearn.metrics import cohen_kappa_score
#print(cohen_kappa_score(y_test, y_pred))